In [ ]:
# Zili
import csv
import numpy as np
import multiprocessing as mp
import random
import matplotlib.pyplot as plt
import pickle
import pandas as pd
#import tensorflow as tf
import json
from sklearn.model_selection import train_test_split
import math

In [1]:
data_dir='data/'

#table='REACT_COVID_Demographics'
#table='REACT_COVID_Demographics_20200506'
#table='REACT_Events'
#table='REACT_LabResults'
#table='REACT_PharmacyData'
#table='REACT_UHSCOVIDTest_processed'
#table='REACT_Vitalsigns_Categorical'
#table='REACT_Vitalsigns_Numeric'

In [2]:
#df=pd.read_csv(data_dir+table+'.csv')
#df.head(20)

In [3]:
#df.info()

Extract AIR_ventilator information from table REACT_Events

In [ ]:
air_ventilator_degree=[
    'Air - Not Supported',
    'Nasal Specs',
    'Face Mask',
    'Venturi Mask',
    'Trachy Mask',
    'Non-Rebreath Mask',
    'Optiflow / Hi Flow',
    'NIV - CPAP nasal mask',
    'NIV - CPAP face mask',
    'NIV - CPAP full face mask',
    'NIV - BIPAP nasal mask',
    'NIV - BIPAP face mask',
    'NIV - BIPAP full face mask',
    'Invasive Ventilation'
]
air_ventilator_degree_dic=dict([(air_ventilator_degree[i],float(i)) for i in range(len(air_ventilator_degree))])
air_ventilator_degree_dic

In [ ]:
df_REACT_Vitalsigns_Categorical=pd.read_csv(data_dir+'REACT_Vitalsigns_Categorical'+'.csv')
#df_REACT_Vitalsigns_Categorical.info()

In [ ]:
studyid_air_ventilator_dic=dict()

for studyid,air_vent in df_REACT_Vitalsigns_Categorical[df_REACT_Vitalsigns_Categorical.VALUE.isin(air_ventilator_degree_dic)][['STUDY_ID','VALUE']].values:
    if studyid not in studyid_air_ventilator_dic or studyid_air_ventilator_dic[studyid]<air_ventilator_degree_dic[air_vent]:
        studyid_air_ventilator_dic[studyid]=air_ventilator_degree_dic[air_vent]

df_studyid_air_vent=pd.DataFrame(
    data=[[key,studyid_air_ventilator_dic[key]] for key in studyid_air_ventilator_dic],
    columns=['STUDY_ID','AIR_VENT_DEGREE']
)
#df_studyid_air_vent

Extract ITU_DAYS information from table REACT_Events

In [ ]:
df_REACT_Events=pd.read_csv(data_dir+'REACT_Events'+'.csv')
#df_REACT_Events.info()

In [ ]:
from datetime import date

def str2date(ss):
    day,month,year=ss.split('/')
    return date(int(year),int(month),int(day))

max_date=max([str2date(x) for x in df_REACT_Events[df_REACT_Events.END_DATE.notnull()].END_DATE.unique()])
max_date=str(max_date.day)+'/'+str(max_date.month)+'/'+str(max_date.year)

#max_date

In [ ]:
studyid_ITU_lst=[]
for studyid,start_date,end_date in df_REACT_Events[df_REACT_Events.EVENT_TYPE=='ITU'][['STUDY_ID','START_DATE','END_DATE']].fillna({'END_DATE':max_date}).values:
    start_day,start_month,start_year=map(int,start_date.split('/'))
    end_day,end_month,end_year=map(int,end_date.split('/'))
    days=int((date(end_year,end_month,end_day)-date(start_year,start_month,start_day)).days)
    studyid_ITU_lst.append([studyid,days])
df_studyid_ITU=pd.DataFrame(data=studyid_ITU_lst,columns=['STUDY_ID','ITU_DAYS'])
#df_studyid_ITU

load table REACT\_COVID\_Demographics

In [ ]:
df_demographics=pd.read_csv(data_dir+'REACT_COVID_Demographics'+'.csv')
#df_demographics.head(10)

In [ ]:
#df_demographics.info()

In [ ]:
#len(df_demographics.STUDY_ID.unique())

In [ ]:
#df_demographics[['STUDY_ID','PATIENT_AGE','GENDER','ETHNIC_GROUP','IS_PREGNANT']]

load table REACT_LabResults

In [ ]:
df_labresults=pd.read_csv(data_dir+'REACT_LabResults'+'.csv')
#df_labresults.head(10)

In [ ]:
#df_labresults.info()

In [ ]:
#len(df_labresults.STUDY_ID.unique())

In [ ]:
testcode_lst=df_labresults.REACT_TESTCODE.unique()
testcode2id=dict([(testcode_lst[i],i) for i in range(len(testcode_lst))])
#testcode_lst

In [ ]:
studyid_textcode_dic=dict()
for studyid,testcode,result in df_labresults[['STUDY_ID','REACT_TESTCODE','PATHOLOGY_RESULT_NUMERIC']].values:
    if studyid not in studyid_textcode_dic:
        studyid_textcode_dic[studyid]=dict()
    if testcode not in studyid_textcode_dic[studyid]:
        studyid_textcode_dic[studyid][testcode]=[]
    studyid_textcode_dic[studyid][testcode].append(result)
#studyid_textcode_dic

In [ ]:
labresults_dic=dict()
for studyid in studyid_textcode_dic:
    if studyid not in labresults_dic:
        labresults_dic[studyid]=[None]*len(testcode_lst)*3
    for testcode in studyid_textcode_dic[studyid]:
        results=studyid_textcode_dic[studyid][testcode]
        labresults_dic[studyid][testcode2id[testcode]*3:(testcode2id[testcode]+1)*3]=[np.min(results),np.max(results),np.mean(results)]
#labresults_dic

In [ ]:
df_studyid_testcode_result=pd.DataFrame(
    data=[[studyid]+labresults_dic[studyid] for studyid in labresults_dic],
    columns=["STUDY_ID"]+[testcode+suffix for testcode in testcode_lst for suffix in ['_min','_max','_mean']]
)
#df_studyid_testcode_result

In [ ]:
#df_studyid_testcode_result.info()

In [ ]:
df_patient_features=df_demographics[['STUDY_ID','PATIENT_AGE','GENDER','ETHNIC_GROUP','IS_PREGNANT']].merge(
    df_studyid_testcode_result,how='inner',on='STUDY_ID'
)
#df_patient_features

In [ ]:
#df_patient_features.info()

Merge dataframes, fill in default value (0.0) for $\textbf{NaN}$ value in columns ITU_DAYS and AIR_VENT_DEGREE

In [ ]:
df_patient_features_labels=df_patient_features.merge(
    df_studyid_ITU,how='left',on='STUDY_ID'
).fillna({'ITU_DAYS':0}).merge(
    df_studyid_air_vent,how='left',on='STUDY_ID'
).fillna({'AIR_VENT_DEGREE':0.0})
#df_patient_features_labels

In [ ]:
df_patient_features_labels.info()

In [ ]:
df_patient_features_labels.columns

X Feature Preprocessing, OneHotEncoding for Categorical features

In [ ]:
X_feature_name_lst=[]

In [ ]:
from sklearn.preprocessing import OneHotEncoder

X_categorical=df_patient_features_labels[['GENDER', 'ETHNIC_GROUP', 'IS_PREGNANT']]

enc = OneHotEncoder(handle_unknown='ignore')
X_onehot=enc.fit_transform(X_categorical).toarray()

X_feature_name_lst+=list(enc.get_feature_names(['GENDER', 'ETHNIC_GROUP', 'IS_PREGNANT']))
#X_feature_name_lst

In [ ]:
numeric_features_lst=[
    'PATIENT_AGE',
    'POTASSIUM_min', 'POTASSIUM_max', 'POTASSIUM_mean', 'ALT_min',
    'ALT_max', 'ALT_mean', 'BILIRUBIN_min', 'BILIRUBIN_max',
    'BILIRUBIN_mean', 'UREA_min', 'UREA_max', 'UREA_mean', 'CREATENINE_min',
    'CREATENINE_max', 'CREATENINE_mean', 'SODIUM_min', 'SODIUM_max',
    'SODIUM_mean', 'CRP_min', 'CRP_max', 'CRP_mean', 'NEUTROPHILS_min',
    'NEUTROPHILS_max', 'NEUTROPHILS_mean', 'EOSINOPHILS_min',
    'EOSINOPHILS_max', 'EOSINOPHILS_mean', 'HB_min', 'HB_max', 'HB_mean',
    'LYMPHOCYTES_min', 'LYMPHOCYTES_max', 'LYMPHOCYTES_mean', 'WBC_min',
    'WBC_max', 'WBC_mean', 'PLATELETS_min', 'PLATELETS_max',
    'PLATELETS_mean', 'GLUCOSE_min', 'GLUCOSE_max', 'GLUCOSE_mean',
    'TRIGYCERIN_min', 'TRIGYCERIN_max', 'TRIGYCERIN_mean', 'FERRITIN_min',
    'FERRITIN_max', 'FERRITIN_mean', 'AST_min', 'AST_max', 'AST_mean',
    'TROPONIN_min', 'TROPONIN_max', 'TROPONIN_mean', 'LDH_min', 'LDH_max',
    'LDH_mean', 'D_DIMER_min', 'D_DIMER_max', 'D_DIMER_mean'
]

X_feature_name_lst+=numeric_features_lst

#X_feature_name_lst

In [ ]:
X_numeric=df_patient_features_labels[numeric_features_lst].values
#X_numeric

In [ ]:
X_missing=np.concatenate([X_onehot, X_numeric], axis=1)
#X_missing

fill in missing features value with mean value

In [ ]:
#from sklearn.impute import KNNImputer

#imputer = KNNImputer(n_neighbors=5, weights="uniform")

from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

X=imputer.fit_transform(X_missing)

#X

Two Y labels ITU_DAYS, AIR_VENTILATOR

In [ ]:
Y_ITU=df_patient_features_labels.values[:,-2]
Y_AIR=df_patient_features_labels.values[:,-1]

ITU_DAYS prediction test, evaluated by R2_score:
$R^2(y, \hat{y}) = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2}$

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

models={
    "Linear":LinearRegression(),
    "MLP":MLPRegressor(max_iter=3000,solver='adam', alpha=1e-3,hidden_layer_sizes=(64, 16)),
    "DecisionTree":DecisionTreeRegressor(),
    "RandomForest":RandomForestRegressor(),
    "AdaBoost":AdaBoostRegressor(n_estimators=100),
    "GradientBoosting":GradientBoostingRegressor(),
}

In [ ]:
from sklearn.model_selection import KFold

from sklearn.metrics import r2_score

r2_result_lst=[]

kf = KFold(n_splits=5)

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y_ITU[train_index], Y_ITU[test_index]
    r2_result_lst.append(
        [r2_score(y_test,models[model_name].fit(X_train,y_train).predict(X_test))
         for model_name in models])

print(pd.DataFrame(r2_result_lst,columns=[model_name for model_name in models]))

ITU_DAYS feature importance analysis $\textbf{RandomForest}$

In [ ]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor().fit(X, Y_ITU)

for feature,importance in zip(X_feature_name_lst,regr.feature_importances_):
    print(feature,":",importance)

In [ ]:
feature_importance_dic=dict(zip(X_feature_name_lst,regr.feature_importances_))
from sklearn.feature_selection import SelectFromModel
sorted(
    [(X_feature_name_lst[i],feature_importance_dic[X_feature_name_lst[i]]) for i in SelectFromModel(regr, prefit=True, max_features=10).get_support(indices=True)],
    key=lambda x:x[1],
    reverse=True
)

ITU_DAYS feature importance analysis $\textbf{AdaBoost}$

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

regr = AdaBoostRegressor().fit(X, Y_ITU)

for feature,importance in zip(X_feature_name_lst,regr.feature_importances_):
    print(feature,":",importance)

In [ ]:
feature_importance_dic=dict(zip(X_feature_name_lst,regr.feature_importances_))
from sklearn.feature_selection import SelectFromModel
sorted(
    [(X_feature_name_lst[i],feature_importance_dic[X_feature_name_lst[i]]) for i in SelectFromModel(regr, prefit=True, max_features=10).get_support(indices=True)],
    key=lambda x:x[1],
    reverse=True
)

ITU_DAYS feature importance analysis $\textbf{GradientBoostingTree}$

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

regr = GradientBoostingRegressor().fit(X, Y_ITU)

for feature,importance in zip(X_feature_name_lst,regr.feature_importances_):
    print(feature,":",importance)

In [ ]:
feature_importance_dic=dict(zip(X_feature_name_lst,regr.feature_importances_))
from sklearn.feature_selection import SelectFromModel
sorted(
    [(X_feature_name_lst[i],feature_importance_dic[X_feature_name_lst[i]]) for i in SelectFromModel(regr, prefit=True, max_features=10).get_support(indices=True)],
    key=lambda x:x[1],
    reverse=True
)

AIR_ventilator prediction test, evaluated by R2_score:
$R^2(y, \hat{y}) = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2}$

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

models={
    "Linear":LinearRegression(),
    "MLP":MLPRegressor(max_iter=3000,solver='adam', alpha=1e-3,hidden_layer_sizes=(64, 16)),
    "DecisionTree":DecisionTreeRegressor(),
    "RandomForest":RandomForestRegressor(),
    "AdaBoost":AdaBoostRegressor(n_estimators=100),
    "GradientBoosting":GradientBoostingRegressor(),
}

In [ ]:
from sklearn.model_selection import KFold

from sklearn.metrics import r2_score

r2_result_lst=[]

kf = KFold(n_splits=5)

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y_AIR[train_index], Y_AIR[test_index]
    r2_result_lst.append(
        [r2_score(y_test,models[model_name].fit(X_train,y_train).predict(X_test))
         for model_name in models])

print(pd.DataFrame(r2_result_lst,columns=[model_name for model_name in models]))

AIR_ventilator feature importance analysis $\textbf{RandomForest}$

In [ ]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor().fit(X, Y_AIR)

for feature,importance in zip(X_feature_name_lst,regr.feature_importances_):
    print(feature,":",importance)

In [ ]:
feature_importance_dic=dict(zip(X_feature_name_lst,regr.feature_importances_))
from sklearn.feature_selection import SelectFromModel
sorted(
    [(X_feature_name_lst[i],feature_importance_dic[X_feature_name_lst[i]]) for i in SelectFromModel(regr, prefit=True, max_features=10).get_support(indices=True)],
    key=lambda x:x[1],
    reverse=True
)

AIR_ventilator feature importance analysis $\textbf{AdaBoost}$

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

regr = AdaBoostRegressor().fit(X, Y_AIR)

for feature,importance in zip(X_feature_name_lst,regr.feature_importances_):
    print(feature,":",importance)

In [ ]:
feature_importance_dic=dict(zip(X_feature_name_lst,regr.feature_importances_))
from sklearn.feature_selection import SelectFromModel
sorted(
    [(X_feature_name_lst[i],feature_importance_dic[X_feature_name_lst[i]]) for i in SelectFromModel(regr, prefit=True, max_features=10).get_support(indices=True)],
    key=lambda x:x[1],
    reverse=True
)

AIR_ventilator feature importance analysis $\textbf{GradientBoostingTree}$

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

regr = GradientBoostingRegressor().fit(X, Y_AIR)

for feature,importance in zip(X_feature_name_lst,regr.feature_importances_):
    print(feature,":",importance)

In [ ]:
feature_importance_dic=dict(zip(X_feature_name_lst,regr.feature_importances_))
from sklearn.feature_selection import SelectFromModel
sorted(
    [(X_feature_name_lst[i],feature_importance_dic[X_feature_name_lst[i]]) for i in SelectFromModel(regr, prefit=True, max_features=10).get_support(indices=True)],
    key=lambda x:x[1],
    reverse=True
)